## Searching Agents

In [9]:
import os
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.react.agent import create_react_agent
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
tools =[TavilySearch()]


In [17]:
tools =[TavilySearch()]
llm = ChatGroq(model="qwen/qwen3-32b")
react_prompt = hub.pull("hwchase17/react") 

In [18]:
print(react_prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [18]:
from typing import List
from pydantic import BaseModel,Field

class JobPosting(BaseModel):
    "Schema for a source used by the Agent"
    url:str =Field(description="The URL of source")

class AgentResponse(BaseModel):
    """Schema for agent response with answer and sources"""
    answer: str = Field(description="The answer provided by the agent")
    sources: List[JobPosting] = Field(default_factory=list,description="The sources used by the agent to generate the answer")


In [19]:

REACT_PROMPT_WITH_FORMAT_INSTRUCTIONS = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question formatted according to format_instructions: {format_instructions}

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [20]:
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate

In [21]:
tools =[TavilySearch()]
llm = ChatGroq(model="qwen/qwen3-32b")
output_parser = PydanticOutputParser(pydantic_object=AgentResponse)
react_prompt_with_format_instructions = PromptTemplate(
    template=REACT_PROMPT_WITH_FORMAT_INSTRUCTIONS,
    input_variables=["input", "agent_scratchpad", "tool_names"],
).partial(format_instructions=output_parser.get_format_instructions())


In [22]:
agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt_with_format_instructions)

In [23]:
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)
extract_output = RunnableLambda(lambda x: x["output"])
parse_output = RunnableLambda(lambda x: output_parser.parse(x))

chain = agent_executor | extract_output | parse_output

In [25]:
result = chain.invoke(
        input={
            "input": "search for 3 job postings for an ai engineer using langchain in the bay area on linkedin and list their details",
        }
    )



> Entering new AgentExecutor chain...
<think>
Okay, the user wants me to find three job postings for an AI engineer using LangChain in the Bay Area on LinkedIn. Let me start by figuring out the best way to approach this.

First, I need to use the tavily_search tool since it's optimized for comprehensive and accurate results, including LinkedIn job postings. The user specified the location as the Bay Area and the role as AI Engineer with LangChain experience. 

I should construct a search query that includes these keywords. Maybe something like "AI Engineer LangChain job postings LinkedIn Bay Area". Adding "site:linkedin.com" might help narrow down the search to LinkedIn specifically. Let me try that.

After performing the search, I'll need to look through the results to find the job postings. The search results should return URLs and snippets. I need to check if the URLs are actual job postings on LinkedIn. Each posting's URL should have a specific structure, like linkedin.com/jobs/.